<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/TwADR_bigtweet_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'TwADR-L.fold-3.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-3.validation.csv",header=None)
valid.head()

,0,1
0,91,jitters
1,93,shooting
2,187,dry mouth
3,189,dizzy
4,230,mania


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-3.test.csv",header=None)
test.head()

,0,1
0,13,insomniac
1,16,couldn't taste anything
2,61,spins
3,85,tremor
4,94,anxiety


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'TwADR-bigtweet.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'to',
 'the',
 'in',
 'rt',
 'of',
 'for',
 'a',
 'you',
 'i',
 'is',
 'on',
 'pain',
 'and',
 'disease',
 'my',
 'mood',
 'me',
 'increased',
 'amp',
 'with',
 'gluten',
 ',',
 'disorder',
 'abnormal',
 'withdrawal',
 'anxiety',
 'your',
 'free',
 'it',
 'have',
 'symptoms',
 'weight',
 'this',
 'out',
 'are',
 'new',
 'like',
 'decreased',
 'disorders',
 '(',
 ')',
 'about',
 '-',
 'not',
 'from',
 'do',
 'glutenfree',
 'up',
 'our',
 'sleeplessness',
 'behavior',
 'what',
 'nt',
 'be',
 'feeling',
 'drug',
 'syndrome',
 'celiac',
 'got',
 'at',
 'now',
 'diseases',
 'or',
 'insomnia',
 'somnolence',
 'finding',
 'disturbance',
 'sleep',
 'blood',
 'so',
 'that',
 'energy',
 'all',
 'we',
 'food',
 'mental',
 'how',
 'right',
 'can',
 'skin',
 'gain',
 'appetite',
 'depression',
 'acute',
 'more',
 'its',
 'feel',
 'via',
 'tip',
 'if',
 'nausea',
 'excitability',
 'body',
 'hyperactive',
 'as'

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.351199,3.507564,0.453837,09:29


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.277373,2.994108,0.511844,09:36


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.049071,2.949777,0.517857,09:22


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.990120,2.911189,0.521875,09:30


In [25]:
learn.save_encoder('TwADR-bigtweet_fold3_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('TwADR-bigtweet_fold3_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.230175,6.394176,0.109244,00:22


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.275671,5.151800,0.159664,00:28


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.777360,4.922297,0.142857,01:04


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.444275,4.585350,0.201681,01:08
1,5.255297,4.423802,0.184874,01:07


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.020309,4.187103,0.260504,01:03
1,4.812740,4.088792,0.235294,01:11


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.603642,3.851578,0.319328,01:06
1,4.430422,3.824541,0.268908,01:06


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.231742,3.599775,0.319328,01:02
1,4.065715,3.564520,0.319328,01:02


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.891010,3.481669,0.378151,01:08
1,3.765811,3.443665,0.369748,01:04


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.594063,3.417758,0.352941,01:05
1,3.498259,3.337436,0.352941,01:09
2,3.315581,3.139872,0.386555,01:05
3,3.150140,3.133406,0.394958,01:03


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.984739,3.152474,0.386555,01:03
1,2.913013,3.065260,0.378151,01:08
2,2.727094,3.031102,0.386555,01:06
3,2.576293,3.021726,0.378151,01:06


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.453200,3.074716,0.361345,01:03
1,2.397531,3.007871,0.436975,01:03
2,2.228101,3.047319,0.386555,01:10
3,2.064041,2.992717,0.403361,01:10


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.974077,2.965919,0.428571,01:07
1,1.939542,3.023756,0.403361,01:03
2,1.797123,3.006583,0.378151,01:02
3,1.630190,2.960045,0.378151,01:08


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.582913,3.066750,0.403361,01:01
1,1.550522,3.058746,0.394958,01:03
2,1.425883,3.029437,0.420168,01:05
3,1.311145,3.032176,0.428571,01:04


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.194652,2.997589,0.420168,01:08
1,1.215578,3.092525,0.403361,01:08
2,1.126699,3.111974,0.403361,01:05
3,1.027162,3.112424,0.394958,01:02


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.939522,3.090885,0.403361,01:07
1,0.938858,3.241427,0.386555,01:08
2,0.909157,3.193712,0.386555,01:07
3,0.859183,3.170118,0.394958,01:03


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.817865,3.329440,0.386555,01:07
1,0.800175,3.263962,0.428571,01:02


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.713798,3.409021,0.369748,01:09
1,0.707168,3.383601,0.394958,01:07


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.668918,3.377844,0.403361,01:09


In [44]:
# save the best model

learn.save_encoder('TwADR-bigtweet_fold3')

In [45]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.616187,3.447662,0.369748,01:08


In [46]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.623968,3.337865,0.428571,01:07


In [47]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.599648,3.464556,0.394958,01:06


In [51]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.617610,3.501396,0.403361,01:07


# Part three: Predict on the test dataset

In [52]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [53]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[393, 1962, 1295, 791, 978, 750, 109, 1295, 189, 1007, 233, 249, 232, 1610, 1727, 1099, 1054, 392, 13, 393, 1787, 791, 1526, 2168, 2063, 232, 1199, 1230, 629, 1483, 1204, 749, 749, 749, 974, 792, 792, 974, 809, 809, 809, 1308, 824, 1610, 1718, 908, 1727, 1507, 974, 974, 974, 232, 1676, 1669, 791, 393, 1295, 1099, 1925, 1136, 1295, 1145, 749, 1204, 791, 1340, 1265, 1062, 1295, 189, 1610, 1597, 1597, 1283, 1508, 749, 1769, 1484, 1787, 561, 561, 1295, 1099, 1295, 418, 1783, 426, 1597, 1610, 1610, 1669, 1521, 1610, 1701, 1526, 1711, 1711, 1295, 1718, 1295, 2191, 1718, 1718, 1767, 2032, 13, 2032, 1787, 1787, 2150, 393, 393, 1787, 1815, 119, 1610, 825, 1526, 825, 1915, 418, 1936, 1936, 749, 1295, 858, 1073, 393, 1787, 1787, 1769, 2081, 1295, 224, 2147, 2147, 2147, 233, 2147, 791, 1295, 561, 2149]
[13, 16, 61, 85, 94, 97, 109, 121, 189, 203, 233, 249, 265, 276, 284, 309, 359, 392, 393, 393, 393, 426, 441, 452, 452, 452, 561, 579, 629, 712, 739, 749, 749, 749, 792, 792, 792, 792, 809, 

In [54]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

46
0.32167832167832167
